In [1]:
import pandas as pd
import numpy as np
import random
import networkx as nx
import bridgeness
from numpy.random import choice, uniform

In [4]:
FN = "graph2.txt.csv"
data = pd.read_csv(FN, delimiter = " ")

G = nx.Graph()

for i,s in data.iterrows():
    u = s[0]
    v = s[1]

    G.add_edge(u,v)
    

In [5]:
#G.nodes()

NodeView((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200))

In [6]:
bri = bridgeness.bridgeness_centrality(G)
#e_bri = bridgeness.edge_bridgeness_centrality(G)
#bet = bridgeness.betweenness_centrality(G)

In [49]:
phi = 100
mu = 0.2
lab0 = 81  #82 in matlab
lab1 = 190 #191 in matlab

adj = nx.to_numpy_matrix(G)
bri_vals = list(bri.values())
quality = (1 / np.exp( np.multiply( bri_vals, phi )))
diag = np.diagflat(quality) #k: v = node, bridgeness
_tmp = np.matmul(diag, adj)
_tmp_sum_on_rows = np.sum(_tmp, axis=0)
_tmp_sum_on_rows_recip = np.reciprocal(_tmp_sum_on_rows)
norm = np.diagflat( _tmp_sum_on_rows_recip )

T = np.matmul( _tmp, norm )

num_nodes = len(G.nodes())
alpha = 2.0/(2.0 + mu)
prob_comm = T 

labels = np.zeros( (num_nodes, 2) )
labels[lab0,0] = 1
labels[lab1,1] = 1

op1 = (1-alpha) * np.eye(num_nodes) 
op2 = ( np.eye(num_nodes) - ( alpha * T))
op = np.dot(op1, op2.I)

probs = np.dot(op, labels)

In [50]:
#pd.DataFrame.to_csv(pd.DataFrame(probs), 'probs.csv')

In [ ]:
#2-comm RW

#start from lab0(s), lab1(s)

steps = 10000000 #max steps
convcheckfreq = 10000 #check if converged every N steps
conv_thr = 1e-06

n0 = lab0
n1 = lab1

labels=[n0,n1]

#visit probability vec
rw_visits = np.zeros( (num_nodes, 2) )
last_rw_visits = np.copy(rw_visits)

nodes = range(num_nodes)

print("Alpha = %.3f" %(alpha))

#COMM0
for s0 in range(1,steps+1):

    #start from
    #go back to labeled node? prob = 1-alpha; prob to trans = alpha
    if (uniform() < alpha): #
        trans_probs = T[:,n0].view(np.ndarray).flatten() 
        trans_to = int( choice(nodes, 1, p = trans_probs) )
        n0 = trans_to
    else:
        n0 = lab0

    rw_visits[n0,0] += 1
    
    if (s0 % convcheckfreq == 0):
        diff_rw_visits = ((rw_visits[:,0]/s0 - last_rw_visits[:,0]/s0)**2).sum()
        print("At step %d, diff %.10f" %(s0, diff_rw_visits) )
        if ((diff_rw_visits) < conv_thr):# and np.all(rw_visits != last_rw_visits)):
            print("Converged at <%.10f" % conv_thr)
            break
        last_rw_visits = np.copy(rw_visits)

#COMM1
for s1 in range(1,steps+1):

    #start from
    #go back to labeled node? prob = 1-alpha; prob to trans = alpha
    if (uniform() < alpha):
        trans_probs = T[:,n1].view(np.ndarray).flatten() 
        trans_to = int( choice(nodes, 1, p = trans_probs) )
        n1 = trans_to
    else:
        n1 = lab1

    rw_visits[n1,1] += 1
    
    if (s1 % convcheckfreq == 0):
        diff_rw_visits = ((rw_visits[:,1]/s1 - last_rw_visits[:,1]/s1)**2).sum()
        print("At step %d, diff %.10f" %(s1, diff_rw_visits) )
        if ((diff_rw_visits) < conv_thr):# and np.all(rw_visits != last_rw_visits)):
            print("Converged at <%.10f" % conv_thr)
            break
        last_rw_visits = np.copy(rw_visits)


Alpha = 0.909
At step 10000, diff 0.0285454800
At step 20000, diff 0.0074642700
At step 30000, diff 0.0034363867
At step 40000, diff 0.0018923038
At step 50000, diff 0.0011974880
At step 60000, diff 0.0008629828
At step 70000, diff 0.0006179037
At step 80000, diff 0.0004638200
At step 90000, diff 0.0003757723
At step 100000, diff 0.0003066400
At step 110000, diff 0.0002462471
At step 120000, diff 0.0002134525
At step 130000, diff 0.0001753608
At step 140000, diff 0.0001564294
At step 150000, diff 0.0001362057
At step 160000, diff 0.0001164367
At step 170000, diff 0.0001019649
At step 180000, diff 0.0000985472
At step 190000, diff 0.0000826402
At step 200000, diff 0.0000747260
At step 210000, diff 0.0000722736
At step 220000, diff 0.0000654343
At step 230000, diff 0.0000586597
At step 240000, diff 0.0000534279
At step 250000, diff 0.0000518699
At step 260000, diff 0.0000458226
At step 270000, diff 0.0000429433
At step 280000, diff 0.0000389196
At step 290000, diff 0.0000366735
At step 3

In [56]:
#comm_cf = ( (probs[:,0]) > (probs[:,1]) )
#comm_rw = ( (rw_visits[:,0]/s0) > (rw_visits[:,1]/s1) )
#print(comm_cf, comm_rw)

#print(rw_visits[:,0]/s0)
#print(rw_visits[:,1]/s1)
#print(sum(rw_visits[:,0]/s0))
#print(sum(rw_visits[:,1]/s1))

[6.26900e-03 0.00000e+00 1.83720e-02 3.49000e-04 3.91100e-03 9.75000e-04
 2.50990e-02 8.30000e-05 0.00000e+00 1.12500e-03 5.46000e-04 3.14760e-02
 1.05400e-03 1.76010e-02 0.00000e+00 1.09640e-02 4.01000e-04 7.90000e-05
 3.66900e-03 5.60000e-05 1.27500e-03 2.27600e-03 1.81800e-02 3.74300e-03
 7.33300e-03 2.76100e-03 2.04610e-02 4.00000e-06 3.06600e-03 2.69360e-02
 3.17800e-02 1.36270e-02 1.33700e-02 3.17000e-04 6.13700e-03 1.28570e-02
 9.62600e-03 1.48000e-03 8.90000e-04 2.62400e-03 2.03000e-04 1.32750e-02
 7.14000e-04 6.57300e-03 2.54430e-02 2.90000e-04 3.07590e-02 9.82000e-03
 8.06000e-04 3.13900e-03 2.29800e-03 1.27100e-03 1.11400e-03 2.36850e-02
 1.90110e-02 5.77000e-04 2.31870e-02 1.00000e-04 4.92700e-03 1.87840e-02
 6.50000e-05 1.74110e-02 9.40000e-05 8.50000e-05 1.89920e-02 1.53300e-02
 9.99000e-04 3.13000e-04 2.84740e-02 2.86630e-02 8.38000e-04 1.81320e-02
 5.72000e-04 3.41600e-03 2.86290e-02 3.27200e-03 3.25000e-04 3.95500e-03
 6.67200e-03 2.16420e-02 5.37500e-03 1.12468e-01 2.

In [16]:
#crosscheck
#matlab_solution = pd.read_csv("solution-matlab-graph2-200nodi-twomoons.txt", delimiter = " ", skipinitialspace=True, header=None)

In [35]:
#compare = pd.concat([matlab_solution, pd.DataFrame(probs)], axis=1)
#compare.columns = ['matlab0', 'matlab1', 'rw0', 'rw1']

#compare['diff0']=compare['matlab0']-compare['rw0']
#compare['diff1']=compare['matlab1']-compare['rw1']
#compare['comm_matlab'] = np.where( (compare['matlab0']>compare['matlab1']), 0, 1)
#compare['comm_rw'] = np.where( (compare['rw0']>compare['rw1']), 0, 1)

In [38]:
#pd.DataFrame.to_csv(pd.DataFrame(compare), 'compare.csv')

In [45]:
#probs
#bri_vals
#op
#pd.DataFrame.to_csv(pd.DataFrame(op), 'operator_py.csv')